In [18]:
# We'll need to install the Redis client
# !pip install redis

#Install wget to pull zip file
# !pip install wget

#Install required packages
# !pip install openai
#!pip install pandas
#!pip install numpy
!pip install python-dotenv

You should consider upgrading via the '/Users/pnagarajan/projects/msft/data-champ/azure-vector-database-samples/.venv/bin/python3 -m pip install --upgrade pip' command.


In [6]:
import openai

from typing import List, Iterator
import pandas as pd
import numpy as np
import os
import wget
from ast import literal_eval

# Redis client library for Python
import redis

# I've set this to our new embeddings model, this can be changed to the embedding model of your choice
EMBEDDING_MODEL = "text-embedding-ada-002"

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [9]:
products_df = pd.read_json('../data/text/product_docs_embeddings.json')

In [10]:
products_df.head()

,id,title,content,category,title_vector,content_vector
0,1,Azure App Service,Azure App Service is a fully managed platform ...,Web,"[-0.0106361033, -0.021644678, 0.0019778875, -0...","[0.007650437800000001, -0.0236263517, 0.012058..."
1,2,Azure Functions,Azure Functions is a serverless compute servic...,Compute,"[-0.0094142705, -0.0047656302, -0.005457249, -...","[-0.014505187000000001, -0.0127168763, -0.0093..."
2,3,Azure Cognitive Services,Azure Cognitive Services are a set of AI servi...,AI + Machine Learning,"[-0.012958467000000001, -0.0041865818, -0.0088...","[-0.0162250493, -0.0012716976, 0.0024742542, -..."
3,4,Azure Storage,"Azure Storage is a scalable, durable, and high...",Storage,"[-0.0216679014, -0.0159060247, 0.0107527971000...","[-0.019703323000000002, -0.0143261477, 0.00300..."
4,5,Azure SQL Database,Azure SQL Database is a fully managed relation...,Databases,"[-0.0105225565, -0.0080567617, -0.0060751964, ...","[-0.0062997062000000005, -0.0119180158, 0.0005..."


In [59]:
%reload_ext dotenv
%dotenv

In [58]:
del os.environ["REDIS_HOST"]
del os.environ["REDIS_PORT"]
del os.environ["REDIS_PASSWORD"]

In [40]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)
import os

REDIS_HOST =  os.environ.get("REDIS_HOST")
REDIS_PORT = os.environ.get("REDIS_PORT")
REDIS_PASSWORD = os.environ.get("REDIS_PASSWORD")
# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    ssl=True,
    password=REDIS_PASSWORD
)
redis_client.ping()

champs.eastus.redisenterprise.cache.azure.net 10000 OEZvhzDzxVbaSwCsRW092+3sNAGL+zHB9jkME6HJJuI=


True

In [41]:
# Constants
VECTOR_DIM = len(products_df['title_vector'][0]) # length of the vectors
VECTOR_NUMBER = len(products_df)                 # initial number of vectors
INDEX_NAME = "embeddings-index"                 # name of the search index
PREFIX = "doc"                                  # prefix for the document keys
DISTANCE_METRIC = "COSINE"                      # distance metric for the vectors (ex. COSINE, IP, L2)

In [42]:
# Define RediSearch fields for each of the columns in the dataset
title = TextField(name="title")
category = TextField(name="category")
content = TextField(name="content")
title_embedding = VectorField("title_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
content_embedding = VectorField("content_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
fields = [title, category, content, title_embedding, content_embedding]

In [43]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )

In [44]:
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    for doc in records:
        key = f"{prefix}:{str(doc['id'])}"

        # create byte vectors for title and content
        title_embedding = np.array(doc["title_vector"], dtype=np.float32).tobytes()
        content_embedding = np.array(doc["content_vector"], dtype=np.float32).tobytes()

        # replace list of floats with byte vectors
        doc["title_vector"] = title_embedding
        doc["content_vector"] = content_embedding

        client.hset(key, mapping = doc)

In [45]:
index_documents(redis_client, PREFIX, products_df)
print(f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Loaded 84 documents in Redis search index with name: embeddings-index


In [71]:
!pip install openai"[datalib]"

You should consider upgrading via the '/Users/pnagarajan/projects/msft/data-champ/azure-vector-database-samples/.venv/bin/python3 -m pip install --upgrade pip' command.


In [72]:
#setup azure open ai
openai.api_type = "azure"
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_BASE_URL")
openai.api_version = "2023-06-01-preview"


def create_embedding(user_query):
    return openai.Embedding.create(input=user_query,
                                   deployment_id='ada-deployment',
                                             model='text-embedding-ada-002',
                                            )["data"][0]['embedding']

In [49]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str = "embeddings-index",
    vector_field: str = "title_vector",
    return_fields: list = ["title", "category", "content", "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
) -> List[dict]:

    # Creates embedding vector from user query
    embedded_query = create_embedding(user_query=user_query)

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    for i, article in enumerate(results.docs):
        score = 1 - float(article.vector_score)
        print(f"{i}. {article.title} (Score: {round(score ,3) })")
    return results.docs

In [73]:
# For using OpenAI to generate query embedding
create_embedding('test')

MissingDependencyError: 

OpenAI error:

    missing `numpy`

This feature requires additional dependencies:

    $ pip install openai[datalib]

